# Introduction

This notebook explores use of Large Language models for automatically creating a data dictionary for consumption data (such as household spending) from country survey data where data may be distributed across many files and meta data provided in documents such as a PDF.

This was done as part of the [IMF 2023 September DataDive](https://www.datakind.org/imf-datadive-event).

# Setup

## Data

You will need to download a file the [World bank](https://microdata.worldbank.org/index.php/catalog/4297/study-description) microdata site for Senegal:

- Documentation PDF (contains data dictionary for each file)

Save into sub-folder `data`.

## Environment

A miniconda environment has been provided, to use this ...

1. Install [miniconda](https://docs.conda.io/en/latest/miniconda.html) by selecting the installer that fits your OS version. Once it is installed you may have to restart your terminal (closing your terminal and opening again)
2. In this directory, open terminal
3. `conda env create -f environment.yml`
4. `conda activate ddenv`

## Libraries and parameters

In [5]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.document_loaders.json_loader import JSONLoader
from langchain.embeddings import OpenAIEmbeddings 
from langchain.vectorstores import Chroma 
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
#from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import langchain
from typing import Dict, Any, Tuple
from langchain.memory.utils import get_prompt_input_key
import shutil
import os
import sys
import json
from googletrans import Translator
translator = Translator()

# A little mod to enable using memory *and* getting docs. See: https://github.com/langchain-ai/langchain/issues/2256#issuecomment-1665188576
def _get_input_output(
    self, inputs: Dict[str, Any], outputs: Dict[str, str]
) -> Tuple[str, str]:
    if self.input_key is None:
        prompt_input_key = get_prompt_input_key(inputs, self.memory_variables)
    else:
        prompt_input_key = self.input_key
    if self.output_key is None:
        output_key = list(outputs.keys())[0]
    else:
        output_key = self.output_key
    return inputs[prompt_input_key], outputs[output_key]

# Annoyingly, and unlike OpenAI direct, some API settings differ depending on service used
def set_azure_env(type="embedding"):
    os.environ["OPENAI_API_TYPE"] = "azure"
    fields = ["OPENAI_API_BASE", "OPENAI_API_KEY"]
    if type == "embedding":
        os.environ["OPENAI_API_VERSION"]=os.getenv("EMBEDDING_API_VERSION")
    else:
        os.environ["OPENAI_API_VERSION"]=os.getenv("CHAT_API_VERSION")
    for k in fields:
        os.environ[k] = os.getenv(k)
  
langchain.memory.chat_memory.BaseChatMemory._get_input_output = _get_input_output



from dotenv import load_dotenv
load_dotenv()

vecs_dir = "data/vecs"
pdf = "data/ddi-documentation-english_microdata-4297.pdf"

# GPT temperature, 0.0 is more deterministic, 1.0 is more random
temperature = 0.0

# Analysis

## Read in meta data from PDF

In [6]:
print(f"Reading the PDF {pdf}...")
loader = PyPDFLoader(pdf)
documents = loader.load()

print("Chunking text into documents ...")
text_splitter = CharacterTextSplitter(separator="\n",chunk_size=2000,chunk_overlap=500,length_function=len)
docs = text_splitter.split_documents(documents)
print("Done")

Reading the PDF data/ddi-documentation-english_microdata-4297.pdf...
Chunking text into documents ...
Done


In [8]:
# Create vector DB directory
if os.path.exists(vecs_dir):
    shutil.rmtree(vecs_dir)
os.makedirs(vecs_dir)

# Azure
print("Setting up embedding model ...")

set_azure_env("embedding")

embedding_model = OpenAIEmbeddings(deployment=os.getenv("EMBEDDING_DEPLOYMENT"), chunk_size=16)

print("Setting up chat model...")

set_azure_env("chat")

chat_model = AzureChatOpenAI(temperature=temperature, deployment_name=os.getenv("CHAT_DEPLOYMENT"))

print("Creating vector DB ...")
vectordb = Chroma.from_documents(docs, embedding=embedding_model,persist_directory=vecs_dir)
vectordb.persist()

# Set up chat
memory = ConversationBufferMemory(memory_key="chat_history", input_key='question', output_key='answer', return_messages=True)
pdf_qa = ConversationalRetrievalChain.from_llm(chat_model, vectordb.as_retriever(search_kwargs={"k": 10}), memory=memory, \
                                                   return_source_documents=True)

Setting up embedding model ...


ValidationError: 2 validation errors for OpenAIEmbeddings
deployment
  none is not an allowed value (type=type_error.none.not_allowed)
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

## FAISS search

Let's do a [FAISS search](https://python.langchain.com/docs/integrations/vectorstores/faiss) to see how it performs.

In [ ]:
db = FAISS.from_documents(docs, embedding_model)

In [ ]:
query = "Which sections relate to household consumption, expenditure?"
faiss_docs = db.similarity_search_with_score(query, k=10)
for d in faiss_docs:
    score = d[1]
    content = d[0].page_content
    print("\n", f"Score: {score} \n{content}")


 Score: 0.3228965401649475 
care.17872 6
s09e_me_sen2018
Section 9: Nonfood consumption
Part E: Nonfood consumption, 6 last months
- This part provides information on expenses made on clothes, shoes, and jewelry purchased for
celebrations and not considered in the other elements above.35635 6
s09f_me_sen2018
Section 9: Nonfood consumption
Part F: Nonfood consumption, 12 last months
- Just as in the previous part, this one also provides information on expenses made on items such
as clothes, shoes, and jewelry purchased for celebrations and not considered in the other elements
above. The section also contains data on utility bills, various house items, cars, motorcycles, and
various maintenance expenses.29697 6

 Score: 0.32371366024017334 
- It collects data on the community’s participation and sense of ownership when it comes to the
construction and maintenance of shared infrastructures such as roads, bridges, schools, health
centers, boreholes/wells, and markets.1291 59
s05_co_sen201

OK, seemed to find the food and non-food sections in the document.

## Chat interface

In [ ]:
def ask_question(
    query: str,
    qa: object,
    output_docs: bool = True,
    preprocess_time_context: bool = False,
    auto_translate: bool = False,
) -> dict:

    print(f"\nQuestion: \n{query}")
    result = qa({"question": query})

    # Translate back to source language
    if auto_translate:
        q = translator.translate(result['answer'], dest='en')
        result['answer'] = q.text
    
    print(f"\nAnswer:\n{result['answer']}")
    if output_docs:
        for doc in result['source_documents']:
            print('\n')
            print(json.dumps(vars(doc), indent=4))
    return result


In [ ]:
## Ask about consumption datasets

In [ ]:
result = ask_question("What datasets are there related to household consumption and spending?", pdf_qa, output_docs=False)


Question: 
What datasets are there related to household consumption and spending?

Answer:
There are several datasets related to household consumption and spending in the Senegal - Enquête Harmonisée sur le Conditions de Vie des Ménages 2018-2019:

1. Data file: ehcvm_conso_sen2018 - Consumption aggregates: This dataset provides information on household consumption aggregates, including the country, year of the survey, household ID, cluster, household identification, region of residence, milieu of residence, zae, domain, household weight, product code, mode of acquisition, and expenditure.

2. Data file: s09a_me_sen2018 - Section 9: Nonfood consumption, Part A: parties and celebrations last 12 months: This dataset assesses nonfood consumption habits and provides data on expenses supported during celebrations. It includes variables such as vague, grappe, household identification, event code, expenses for festivities, total expenditure on food, total expenditure on beverages, total expe

### Automatically create JSON data dictionary

In [ ]:
q = '''
    For each file related to household consumption and spending, find the fields and their definitions.
'''

result = ask_question(q, pdf_qa, output_docs=False)



Question: 

    For each file related to household consumption and spending, find the fields and their definitions.




Answer:
Here are the fields and their definitions for each file related to household consumption and spending:

1. Data file: s09a_me_sen2018
   - Section 9: Nonfood consumption
   - Part A: Parties and celebrations last 12 months
   - Variables:
     - V428: Vague
     - V429: Grappe
     - V430: Identifiant du ménage
     - V431: 9A.01. Code événement
     - V432: 9A.02. 12 derniers jours, avez-vous effectué des dépenses pour les fêtes?
     - V433: 9A.03. Quel est le montant total de cette dépense en alimentation?
     - V434: 9A.04. Quel est le montant total de cette dépense en boissons?
     - V435: 9A.05. Mtt total de cette dépense en habits, chaussures, coiffure, et bijoux
     - V436: 9A.06. Mtt total d cette dép en location salles, chaises, et autres locations
     - V437: 9A.07. Quel est le montant total des autres dépenses non alimentaires ?
     - V438: 9A.00. Code ID du principal répondant

2. Data file: s09b_me_sen2018
   - Section 9: Nonfood consumption
   - Part B: Non

### Do the same, with auto-translation

In [ ]:
result = ask_question(q, pdf_qa, output_docs=False, auto_translate=True)


Question: 

    For each file related to household consumption and spending, find the fields and their definitions.


Answer:
Here are the fields and their definitions for each file related to household consumption and expenditure:

File s09a_me_sen2018 (Part A: holidays and celebrations of the last 12 months):
- V428: Wave
- V429: Cluster
- V430: Household identifier
- V431: Event code
- V432: Have you made any holiday expenses in the last 12 days?
- V433: Total amount of food expenditure
- V434: Total amount of beverage expenditure
- V435: Total amount of expenditure on clothes, shoes, hairstyle and jewelry
- V436: Total amount of expenditure on rental of rooms, chairs and other rentals
- V437: Total amount of other non-food expenses
- V438: ID code of the main respondent

File s09b_me_sen2018 (Part B: non-food consumption - last 7 days):
- V439: Wave
- V440: Cluster
- V441: Household identifier
- V442: Product/Service Code
- V443: Your MEN purchased or picked up [PRODUCT/SERVICE] i

Does it work? Maybe, maybe not. This technique would need confirmation and refiement for it to be robust. Testing it on other countries is key.

# Some possible next steps

- Very close testing, perhaps different prompt result in better stability/results
- Chunking and overlap when creating text for embedding may play a role. Subsequent LLM summarization needs to know surrounding context
- Other PDF formats, need to be tested
- Extending for other files, JSON
- The number of records returned in the semantic search stage is fixed to 10. But perhaps the initial FAISS query could figure out what the true number of results should be. Token limits may become an issue here.